In [1]:
import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from GameDayFunctions.fangraphs_projection_2020 import Projection
#from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer')

In [285]:
class Draft:

    def __init__(self, projections_object, 
                 draft_position = 6,
                 number_teams = 12,
                 roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5},
                 batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS'],
                 pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] ):

        self.number_rounds = sum(roster_spots.values())
        self.draft_position = draft_position
        self.round_number = 0
        self.draft_number = 0
        self.player_projections = projections_object
        self.remaining_ranked_players = projections_object.all_rank 
        #self.roto_standings = []
        self.roto_stats_batting = pd.DataFrame(columns =  batter_stats[1:])
        self.roto_stats_pitching = pd.DataFrame(columns =  pitcher_stats[1:])
        self.teams = {}
        for i in np.arange(number_teams):
            roto_stats = {}
            roto_stats['batting_stats'] = pd.DataFrame(columns =  batter_stats)
            roto_stats['pitching_stats'] = pd.DataFrame(columns =  pitcher_stats)
            roto_stats['roster_spots'] = roster_spots.copy()
            roto_stats['roster'] = {}
            self.teams[i] = roto_stats
    
    # Find Resulting Standings
    def tabulate_roto(self, teams):
        roto_stats_batting = self.roto_stats_batting.copy()
        roto_stats_pitching = self.roto_stats_pitching.copy()
        for iteam in np.arange(len(teams)):
            raw_team_batting = teams[iteam]['batting_stats']
            raw_team_pitching = teams[iteam]['pitching_stats']
            roto_team_batting = raw_team_batting.mean()
            roto_team_pitching = raw_team_pitching.mean()
            if 'AVG' in raw_team_batting:
                roto_team_batting['AVG'] = (raw_team_batting['AVG']*raw_team_batting['AB']).sum()/raw_team_batting['AB'].sum()
            if 'OPS' in raw_team_batting:
                roto_team_batting['OPS'] = (raw_team_batting['OPS']*raw_team_batting['AB']).sum()/raw_team_batting['AB'].sum() 
            if 'ERA' in raw_team_pitching:
                roto_team_batting['ERA'] = (raw_team_pitching['ERA']*raw_team_pitching['IP']).sum()/raw_team_pitching['IP'].sum() 
            if 'WHIP' in raw_team_pitching:
                roto_team_batting['WHIP'] = (raw_team_pitching['WHIP']*raw_team_pitching['IP']).sum()/raw_team_pitching['IP'].sum()
            batting_stat_names = self.roto_stats_batting.columns.values.tolist()
            pitching_stat_names = self.roto_stats_pitching.columns.values.tolist()
            roto_stats_batting = roto_stats_batting.append(roto_team_batting[batting_stat_names],ignore_index = True)
            roto_stats_pitching = roto_stats_pitching.append(roto_team_pitching[pitching_stat_names],ignore_index = True)
            
        roto_team_stats = pd.concat([roto_stats_batting, roto_stats_pitching], axis=1, sort=False)
        roto_standings_avgs = pd.concat([roto_stats_batting.rank(ascending=False), roto_stats_pitching.rank(ascending=False).rename(columns={"BB": "BBP"})], axis=1, sort=False)
        roto_standings_cnts = pd.concat([roto_stats_batting.rank(), roto_stats_pitching.rank().rename(columns={"BB": "BBP"})], axis=1, sort=False)
        avg_stats = ['CS','BBP','ERA','WHIP','BSV']
        for avg_stat in avg_stats:
            if avg_stat in roto_standings_avgs:
                roto_standings_cnts[avg_stat] = roto_standings_avgs[avg_stat]
        roto_standings = roto_standings_cnts.sum(axis=1).sort_values(ascending=False)
        roto_placement = roto_standings.index.get_loc(self.draft_position)
        #pdb.set_trace()
        return roto_stats_batting, roto_stats_pitching, roto_standings, roto_placement
     
    def confirm_selection(self, teams, team_key, selected_player):
        # Check if Position is a Pitcher or Batter
        if selected_player.EligiblePosition.str.contains('P').bool() == True:
            idx_player = self.player_projections.pitchers_stats.Name.values == selected_player.iloc[0].PLAYER
            statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
            teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline)

        else:
            idx_player = self.player_projections.hitters_stats.Name.values == selected_player.iloc[0].PLAYER
            statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
            teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline)
        return teams
    
    # Do the entire draft one round at a time
    def draft_all(self):
        for iround in np.arange(self.number_rounds):
            self.draft_round(iround)
            self.round_number += 1
            #pdb.set_trace()
        self.roto_stats_batting, self.roto_stats_pitching, self.roto_standings, self.roto_placement = self.tabulate_roto(self.teams)
      
    def draft_remaining(self, teams):
        for iround in range(self.round_number,self.number_rounds):
            #self.draft_round(iround)
            
            draft_order = np.arange(len(self.teams))
            # Reverse draft order every other round
            if iround % 2 == 1:
                draft_order = draft_order[::-1]
            
            if iround == self.round_number:
                draft_order = draft_order[self.draft_position + 1:] # wont work if draw first or last picks...
                
            for iteam in draft_order:
                teams = self.draft_next_best(iteam, teams)

        return teams
        
    
    def draft_round(self, round_key):
        
        # Reverse draft order every other round
        draft_order = np.arange(len(self.teams))
        if round_key % 2 == 1:
            draft_order = draft_order[::-1]
        
        # Reset draft_number at start of round
        self.draft_number = len(self.teams) - len(draft_order)
        
        # Draft each round one team at a time
        for iteam in draft_order:
            
            # When team is draft_position, search for best pick.
            if iteam == self.draft_position:
                best_pick = self.find_best_pick(iteam)
                self.teams, self.remaining_ranked_players = self.draft_next_best(iteam, force_pick = best_pick)
            else: 
                self.teams, self.remaining_ranked_players = self.draft_next_best(iteam)
            #self.teams, self.remaining_ranked_players = self.draft_next_best(iteam)
            self.draft_number += 1
            
        #pdb.set_trace()
    
    def find_best_pick(self, team_key):
        # find_best_pick returns iloc, the index (of df) of the optimal pick
        
        df = self.remaining_ranked_players
        
        # Find out which positions still need filling
        # If UTIL is unfilled, all fielding positions are options.  
        if self.teams[team_key]['roster_spots']['UTIL'] > 0:
            ups = 'C|1B|2B|3B|SS|OF'
            if self.teams[team_key]['roster_spots']['SP'] > 0:
                ups = ups+'|SP'
            if self.teams[team_key]['roster_spots']['RP'] > 0:
                ups = ups+'|RP'
            unfilled_positions = ups.split('|')
        else: 
            unfilled_positions = [k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]
            ups = '|'.join(unfilled_positions)
        print(ups)  
        
        #pdb.set_trace()
        # Begin with next best player.        
        idx_eligible = [0]
        
        # Find index of best player at each remaining position
        for iunfilled in unfilled_positions:
            # WANT TO PUT GOOD PLAYERS IN UTIL, BUT NOT BN            
            if (iunfilled != 'UTIL') & (iunfilled != 'BN'):
                idx_position = [i for i, val in enumerate(df.EligiblePosition.str.contains(iunfilled)) if val] 
                filled = False
                jdx = 0
                while filled == False:
                    if idx_position[jdx] in idx_eligible:
                        jdx+=1
                    else:
                        idx_eligible.append(idx_position[jdx])
                        filled = True
        
        # Get rid of doubles (1B and OF is particularly prone)
        idx_eligible = np.unique(idx_eligible)
        print('Picking from:')
        print(df.iloc[idx_eligible])
        #pdb.set_trace()
        
        #################################
        # START OF LOOP TO FIND BEST PLAYER
        player_based_draft_outcomes = {}

        # Loop over eligible players, then finish the draft
        for iposition, icounter in zip(idx_eligible, range(len(idx_eligible))):
            #pick_ok = False
            
            # make a copy of teams to finish drafting
            teams = self.teams.copy()
            df_copy = df.copy()

            # Check if next best player spot is taken
            #is_eligibile = False
            #check_eligible = df.iloc[idx_eligible[icounter]].EligiblePosition.split('/')
            #for icheck in check_eligible:
            #    if self.teams[team_key]['roster_spots'][icheck] > 0:
            #        is_elegible = True
                    
            df_copy,drafted_player=df_copy.drop(df_copy.iloc[iposition:iposition+1].index),df_copy.iloc[iposition:iposition+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]      
            if eligible_positions == 'Util':
                open_positions = [teams[team_key]['roster_spots']['UTIL']]
            else:
                open_positions = [teams[team_key]['roster_spots'][i] for i in eligible_positions.split('/')]
            
            pdb.set_trace()
            # If the first value is UTIL or BN 
            #if is_elegible == False:
            if np.sum(open_positions) == 0:

                # Check if there are UTIL positions left
                #df_copy,drafted_player=df.drop(df.iloc[0:1].index),df.iloc[0:1]
                #eligible_positions = drafted_player.EligiblePosition.values[0]
                if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                    if self.teams[team_key]['roster_spots']['P'] > 0:
                        position = 'P'
                        df = df_copy
                    elif self.teams[team_key]['roster_spots']['BN'] > 0:
                        position = 'BN'
                        df = df_copy
                    else:
                        position = 'No Roster Spots Left'
                        pick_ok = False
                        print("Something is wrong")
                        pdb.set_trace()
                else:
                    if self.teams[team_key]['roster_spots']['UTIL'] > 0:
                        position = 'UTIL'
                        df = df_copy
                    elif self.teams[team_key]['roster_spots']['BN'] > 0:
                        position = 'BN'
                        df = df_copy
                    else:
                        position = 'No Roster Spots Left'
                        #pick_ok = False
                        print("Something is wrong")
                        pdb.set_trace()

                #print(self.teams[team_key]['roster_spots'])
                #print('Considering '+drafted_player.PLAYER+' for '+position)
            else:
                #pick = idx_eligible[0]
                #df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
                #eligible_positions = drafted_player.EligiblePosition.values[0]

                # For those eligible for multiple positions, split them up and pick one
                #tf_position = [p in eligible_positions for p in unfilled_positions]
                #idx_position = [i for i, val in enumerate(tf_position) if val] 
                position = unfilled_positions[iposition]
                #pick_ok = True

            #if pick_ok == True:
            #teams = self.confirm_selection(teams, team_key, selected_player)
            # Check if Position is a Pitcher or Batter
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline)

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline)

            # LOOP OVER WHOLE REST OF THE DRAFT HERE...
            pdb.set_trace()
            teams = self.draft_remaining(teams)
            
            # Calculate the best pseudo-standings
            pseudo_standings = self.tabulate_roto(teams)
            
            # Decide best choice
            # LOGIC...
            
            # Store the result.  
            #player_based_draft_outcomes = {idx_player:}
            
        # Decide on best choice and return 
        best_pick = 5
        
        return best_pick
        # END OF LOOP TO FIND BEST PLAYER
        #################################
    
    # Strategy is to take the best possible player, even if that means putting them in UTIL or BN (maybe BN should reconsidered...)
    def draft_next_best(self, team_key, teams = 0, df = 0, force_pick = False):
        
        if teams == 0:
            teams = self.teams
        
        if df == 0:
            df = self.remaining_ranked_players

        if (force_pick == False):
            pick_ok = False
            idf = 0
            while pick_ok == False:
                df_copy = df.copy()
                pick_ok = True
                #print('Round '+str(self.round_number))

                # Figure out what positions are still unfilled.
                unfilled_positions = [k for (k,v) in teams[team_key]['roster_spots'].items() if v > 0]
                unfilldpns = '|'.join(unfilled_positions)
                #print(unfilldpns)

                # Find indices of all remaining players in positions of need.
                idx_eligible = [i for i, val in enumerate(df_copy.EligiblePosition.str.contains(unfilldpns)) if val]

                # 
                df_copy,drafted_player=df_copy.drop(df_copy.iloc[idf:idf+1].index),df_copy.iloc[idf:idf+1]
                eligible_positions = drafted_player.EligiblePosition.values[0]

                if eligible_positions == 'Util':
                    open_positions = [teams[team_key]['roster_spots']['UTIL']]
                else:
                    open_positions = [teams[team_key]['roster_spots'][i] for i in eligible_positions.split('/')]
                
                # If there are no SP, RP, or Fielding Positions left, try P, UTIL, or BN
                if np.sum(open_positions) == 0:
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        if teams[team_key]['roster_spots']['P'] > 0:
                            position = 'P'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False
                            
                    else:
                        if teams[team_key]['roster_spots']['UTIL'] > 0:
                            position = 'UTIL'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False

                else:
                    #pick = idx_eligible[0]
                    #df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
                    df = df_copy
                    eligible_positions = drafted_player.EligiblePosition.values
                    #print(eligible_positions)
                    #pdb.set_trace()
                    for single_position in eligible_positions[0].split('/'):
                        if single_position == 'C':
                            chosen_position = 'C'
                            break
                        elif single_position == '2B':
                            chosen_position = '2B'
                            break
                        elif single_position == 'SS':
                            chosen_position = 'SS'
                            break
                        elif single_position == 'OF':
                            chosen_position = 'OF'
                            break
                        elif single_position == '3B':
                            chosen_position = '3B'
                            break
                        else:
                            chosen_position = single_position
                    #pdb.set_trace()

                    # For those eligible for multiple positions, split them up and pick one
                    # could be better...
                    tf_position = [p in chosen_position for p in unfilled_positions]
                    idx_position = [i for i, val in enumerate(tf_position) if val] 
                    position = unfilled_positions[idx_position[0]]
                    pick_ok = True

                if pick_ok == True:
                    # Check if Position is a Pitcher or Batter
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                        teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline)

                    else:
                        idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                        teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline)
                    #print('Team '+ str(team_key) +' Drafting '+drafted_player.PLAYER+' for '+position)
                else:
                    #print('Not Drafting '+drafted_player.PLAYER+' for '+position)
                    # Try while loop again after incrementing idf
                    idf += 1

        else:
            pdb.set_trace()
            df,drafted_player=df.drop(df.iloc[force_pick:force_pick+1].index),df.iloc[force_pick:force_pick+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]

            # For those eligible for multiple positions, split them up and pick one
            tf_position = [p in eligible_positions for p in unfilled_positions]
            idx_position = [i for i, val in enumerate(tf_position) if val] 
            position = unfilled_positions[idx_position[0]]
            
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][teams[team_key]['pitching_stats'].keys()]
                teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline)

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][teams[team_key]['batting_stats'].keys()]
                teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline)        
                
            print('Team '+ str(team_key) +' Drafting '+drafted_player.PLAYER+' for '+position)
        
        # Eliminate one roster_spot
        teams[team_key]['roster_spots'][position] -= 1

        # Add Player to Roster
        if position in self.teams[team_key]['roster']:
            teams[team_key]['roster'][position] = [teams[team_key]['roster'][position], drafted_player.PLAYER.values[0]]
        else:
            teams[team_key]['roster'][position] = drafted_player.PLAYER.values[0]
            
        return teams, df
                    

In [286]:
zips = Draft(player_ranking_zips, number_teams = 12)
steam = Draft(player_ranking_steam, number_teams = 12)

In [288]:
zips.draft_all()

C|1B|2B|3B|SS|OF|SP|RP
Picking from:
                PLAYER EligiblePosition
Rank                                   
13        Jacob deGrom               SP
14    Francisco Lindor               SS
15     Whit Merrifield            2B/OF
16    Charlie Blackmon               OF
20        Max Scherzer               SP
25       Rafael Devers               3B
27       Roberto Osuna               RP
45         DJ LeMahieu         1B/2B/3B
185      J.T. Realmuto                C
> <ipython-input-285-ec8ac03f03d5>(197)find_best_pick()
-> if np.sum(open_positions) == 0:


(Pdb)  cont


> <ipython-input-285-ec8ac03f03d5>(255)find_best_pick()
-> teams = self.draft_remaining(teams)


(Pdb)  team


*** NameError: name 'team' is not defined


...

{'batting_stats':         AB    R  1B  2B 3B  HR  RBI  SB  BB    AVG    OPS
15998  547  108  84  32  4  40  120  16  88  0.293  0.977
15998  547  108  84  32  4  40  120  16  88  0.293  0.977, 'pitching_stats':       IP   W  L CG SHO SV  BB   SO   ERA  WHIP BSV
1  184.3  12  6  1   1  0  42  223  2.88  1.04   0, 'roster_spots': {'C': 1, '1B': 1, '2B': 1, '3B': 1, 'SS': 1, 'OF': 3, 'UTIL': 1, 'SP': 2, 'RP': 2, 'P': 3, 'BN': 5}, 'roster': {}}


(Pdb)  teams[7]


{'batting_stats':         AB    R  1B  2B 3B  HR  RBI  SB  BB    AVG    OPS
15998  547  108  84  32  4  40  120  16  88  0.293  0.977
15998  547  108  84  32  4  40  120  16  88  0.293  0.977, 'pitching_stats': Empty DataFrame
Columns: [IP, W, L, CG, SHO, SV, BB, SO, ERA, WHIP, BSV]
Index: [], 'roster_spots': {'C': 1, '1B': 1, '2B': 1, '3B': 1, 'SS': 1, 'OF': 2, 'UTIL': 1, 'SP': 2, 'RP': 2, 'P': 3, 'BN': 5}, 'roster': {'OF': 'Cody Bellinger'}}


(Pdb)  exit


BdbQuit: 

In [232]:
zips.teams[1]

{'batting_stats':         AB    R   1B  2B 3B  HR  RBI  SB  BB    AVG    OPS
 13611  608  126  101  39  5  34   97  21  79  0.294  0.918
 16252  579   98  104  31  8  20   70  44  50  0.282  0.807
 15149  582   83   95  30  3  31   92   1  51  0.273  0.832
 15149  582   83   95  30  3  31   92   1  51  0.273  0.832
 16997  558   94   92  26  1  41  115   7  52  0.287  0.905
 16997  558   94   92  26  1  41  115   7  52  0.287  0.905
 5235   428   64   77  25  1  19   66   3  44  0.285  0.847
 19293  414   59   66  25  4  13   52  14  38  0.261  0.761
 5930   472   58   92  28  1   9   67   1  50  0.275  0.741
 11609  402   54   60  21  2  19   65   3  46  0.254  0.800,
 'pitching_stats':         IP   W  L CG SHO  SV  BB   SO   ERA  WHIP BSV
 15   167.7  11  6  1   1   0  41  201  3.27  1.07   0
 542   39.7   2  2  0   0  17  15   51  3.40  1.13   1
 41   179.7  13  8  1   1   0  35  172  3.91  1.12   0
 425   50.7   3  2  0   0   0  12   52  3.37  1.12   0
 445   71.0   4  3  0   0   0

In [227]:
zips.draft_all()

Round 0
OF
Rank
1    Team 0 Drafting Christian Yelich for OF
Name: PLAYER, dtype: object
Round 0
OF
Rank
2    Team 1 Drafting Mookie Betts for OF
Name: PLAYER, dtype: object
Round 0
OF
Rank
3    Team 2 Drafting Mike Trout for OF
Name: PLAYER, dtype: object
Round 0
OF
Rank
4    Team 3 Drafting Ronald Acuña for OF
Name: PLAYER, dtype: object
Round 0
OF
Rank
5    Team 4 Drafting Juan Soto for OF
Name: PLAYER, dtype: object
Round 0
3B
Rank
6    Team 5 Drafting Anthony Rendon for 3B
Name: PLAYER, dtype: object
Round 0
1B/OF
Rank
7    Team 6 Drafting Cody Bellinger for 1B
Name: PLAYER, dtype: object
Round 0
1B
Rank
8    Team 7 Drafting Freddie Freeman for 1B
Name: PLAYER, dtype: object
Round 0
3B/SS
Rank
9    Team 8 Drafting Alex Bregman for 3B
Name: PLAYER, dtype: object
Round 0
3B
Rank
10    Team 9 Drafting Nolan Arenado for 3B
Name: PLAYER, dtype: object
Round 0
SP
Rank
11    Team 10 Drafting Justin Verlander for SP
Name: PLAYER, dtype: object
Round 0
OF
Rank
12    Team 11 Drafting J.D. M

In [7]:
steam.draft_all()

In [8]:
steam.roto_placement

9

In [8]:
zips.roto_standings

0     171.0
9     163.0
7     156.0
1     144.5
10    143.0
4     123.0
6     117.0
11    114.5
3     114.0
8     108.0
2     103.5
5     102.5
dtype: float64

In [9]:
steam.roto_standings

0     180.0
9     162.0
10    153.0
11    140.5
1     130.0
4     125.0
7     122.0
3     122.0
2     116.5
6     115.0
5     114.0
8      80.0
dtype: float64

In [10]:
steam.teams[6]['roster']

{'1B': 'Cody Bellinger',
 'SS': 'Fernando Tatis',
 '2B': 'Ozzie Albies',
 'RP': ['Aroldis Chapman', 'Tyler Duffey'],
 'UTIL': 'Carlos Santana',
 'SP': ['Drew Pomeranz', 'Brandon Woodruff'],
 'P': [['Zack Britton', 'John Brebbia'], 'Colin Poche'],
 'BN': [[[['Yoan López', 'Keynan Middleton'], 'Mark Melancon'], 'Luis Patiño'],
  'Tyler Rogers'],
 '3B': 'Luis Arraez',
 'OF': [['Nick Senzel', 'Willie Calhoun'], 'Franmil Reyes'],
 'C': 'Carson Kelly'}